In [1]:
import pandas as pd
import numpy as np

from string import digits

# import nltk
from nltk import word_tokenize
from nltk.util import ngrams

In [2]:
# only for loading in text from a word file
def get_text(doc_name):
    remove_digits = str.maketrans('', '', digits)
    paragraphs = docx.Document(doc_name).paragraphs
    par_text = [paragraph.text.replace('\xa0', ' ') for paragraph in paragraphs]
    text = par_text[0]
    for i in range(1, len(par_text)):
        par = par_text[i]
        if par:
            text += ' ' + par
    return text.lower().translate(remove_digits)

In [3]:
test_doc = get_text('Test.docx')

In [4]:
def neg(bucket):
    if bucket < 4:
        return bucket + 4
    else:
        return bucket - 4
    
def remove_nan(l):
    return [x for x in l if not pd.isnull(x)]

def bin_of_words(filename):
    df = pd.read_excel(filename)
    return [sorted(list(set(remove_nan(df[col].values)))) for col in df.columns]

def sentiment_scores(document):
    bins_of_words = bin_of_words('BINS - version 2.xlsx')
    negators = ['no', 'not', 'lack', 'never', 'none', 'neither', 'nobody', 'few', 'hardly',\
            'little', 'rarely', 'scarcely', 'seldom',]
    i = 0    
    
    word_matrix = np.empty((1, len(bins_of_words)), dtype=object)
    for j in range(len(bins_of_words)):
        if str(word_matrix[i][j]) == 'None':
            word_matrix[i][j] = []
        bucket = bins_of_words[j]
        for word in bucket:
            if word in document:
                for _ in range(document.count(word)):
                    tmp = word_matrix[i][j].copy()
                    tmp.append(word)
                    word_matrix[i][j] = tmp

    negated = []
    token = word_tokenize(document)
    bigrams = ngrams(token,2)
    trigrams = ngrams(token,3)
    for gram in bigrams:
        if gram[0] in negators:
            negated.append(gram[1])
    for gram in trigrams:
        if gram[0] in negators:
            negated.append(gram[2])
            negated.append(gram[1] + ' ' + gram[2])
            
    rows = [0,4]
    for word in negated:
        for j in rows:
            if word in word_matrix[i][j]:
                tmp = word_matrix[i][j].copy()
                tmp.pop(tmp.index(word))
                word_matrix[i][j] = tmp

                j2 = neg(j)
                tmp2 = word_matrix[i][j2].copy()
                tmp2.append(word)
                word_matrix[i][j2] = tmp2 
                break
                
    score_matrix = np.zeros((1, len(bins_of_words)))
    for j in range(len(bins_of_words)):
        score_matrix[i][j] = len(word_matrix[i][j])
    
    positive = score_matrix[i][0]
    negative = score_matrix[i][4]
    total_sentiment = positive - negative

    economic = score_matrix[i][1] 
    health = score_matrix[i][2] 
    environment = score_matrix[i][3]
    total_topic =  economic + health + environment
    if total_sentiment > 0:
        sen_score = total_sentiment/positive
    elif total_sentiment < 0:
        sen_score = total_sentiment/negative
    elif total_sentiment == 0:
        sen_score = 0

    sentiment_score = round(sen_score / 2,2)
    total_economic = round(economic/total_topic,2)
    total_health = round(health/total_topic,2)
    total_environmental = round(environment/total_topic,2)
    return sentiment_score, total_economic, total_health, total_environmental

In [7]:
sentiment, economic, health, environment = sentiment_scores(test_doc)
print(" Sentiment score = {}\n".format(sentiment))
print("  Ecocomic score = {}".format(economic))
print("    Health score = {}".format(health))
print("Evironment score = {}".format(environment))

 Sentiment score = 0.01

  Ecocomic score = 0.32
    Health score = 0.54
Evironment score = 0.14


Bins of words column structure

1: Dream 

2: Economic

3: Health

4: Environment

5: Nightmare